In [1]:
from google.colab import drive
from os.path import join
import os

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/Colab Notebooks/'       # path to your project on Drive



drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)


Mounted at /content/drive


In [ ]:
GIT_USERNAME = "ahhuang007" # replace with yours
GIT_TOKEN = os.environ['git_key_wsb']           # definitely replace with yours
GIT_REPOSITORY = "wallstreetbets"      # ...nah


In [16]:
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already   
%cd "{PROJECT_PATH}"

mkdir: cannot create directory ‘/content/drive/My Drive/Colab Notebooks/wallstreetbets’: File exists
/content/drive/My Drive/Colab Notebooks/wallstreetbets


In [32]:
#GIT_PATH = "https://GIT_TOKEN@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
!git clone "https://$git_key_wsb@github.com/ahhuang007/wallstreetbets.git"
#!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./

Cloning into 'wallstreetbets'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (362/362), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 362 (delta 202), reused 293 (delta 133), pack-reused 0
Receiving objects: 100% (362/362), 3.16 MiB | 10.17 MiB/s, done.
Resolving deltas: 100% (202/202), done.


In [33]:
%cd wallstreetbets

/content/drive/MyDrive/Colab Notebooks/wallstreetbets


In [19]:
!mv ./wallstreetbets/* "{PROJECT_PATH}"

In [31]:
!rm -rf ./wallstreetbets

Pulling data from Github

In [ ]:
!git pull origin

Adding stuff to Github

In [56]:
!git add .

In [57]:
!git config --global user.email "ahhuang007@gmail.com"
!git config --global user.name "ahhuang007"

In [58]:
!git commit -m "figured out nans are from gradient blowup, still need to tune though"

[main 9121ba3] figured out nans are from gradient blowup, still need to tune though
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite colab_wsb.ipynb (75%)


In [43]:
!git remote set-url origin https://$git_key_wsb@github.com/ahhuang007/wallstreetbets.git

In [59]:
!git push origin

Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 1.51 KiB | 386.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/ahhuang007/wallstreetbets.git
   c93d337..9121ba3  main -> main


In [45]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   colab_wsb.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


Actual Code Below

The reason for this Colab file is to explore the viability of using a NN instead of an RL policy - in theory, we already know a basic optimal policy (buy before it increases, sell before it decreases), we just need to figure out if it will increase/decrease beforehand, which a NN can maybe do.

In [2]:
import pandas as pd

In [3]:
%cd drive/MyDrive/Colab Notebooks/wallstreetbets

/content/drive/MyDrive/Colab Notebooks/wallstreetbets


In [4]:
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np

class wsb_dataset(Dataset):
  """class for formatting training data"""

  def __init__(self, df, labels):
    self.df = df
    self.labels = labels
  
  def __len__(self):
    return len(self.df)
  
  def __getitem__(self, idx):
    sample = self.df.iloc[idx, :]
    sample = np.array([sample])
    sample = sample.astype('float')#.reshape(-1, 2)
    label = self.labels[idx]
    return sample, label

In [205]:
#Gathering data, formatting it for NN
from sklearn.preprocessing import MinMaxScaler

dfs = []
training_datas = []
cryptos = ['AAVE', 'ADA', 'ALGO', 'ATOM', 
           'AVAX', 'BCH', 'BTC', 'DOT', 
           'ETH', 'LINK', 'LRC', 'LTC', 
           'MANA', 'MATIC', 'SOL', 'UNI']
cryptos = ['BTC']
for c in cryptos:
    df = pd.read_csv('./data/' + c + '_data.csv')
    '''I think we'll do a similar plan to my source idea.
    5 months for training, 2 months for validation/tuning, 5 months for testing
    '''
    #dfs.append(df[38:175200].reset_index(drop = True))
    #Splitting data into training/validation/test
    
    df = df.drop(['timestamp', 'Unnamed: 0'], axis = 1)
    df = df[38:]

    #Normalizing data
    scaler = MinMaxScaler()
    df = pd.DataFrame(data = scaler.fit_transform(df))
    labels = df[3][1:].reset_index(drop = True)
    training = wsb_dataset(df[:175200].reset_index(drop = True), labels[:175200].reset_index(drop = True))
    validation = wsb_dataset(df[175200:282600].reset_index(drop = True), labels[175200:282600].reset_index(drop = True))
    testing = wsb_dataset(df[262800:-1].reset_index(drop = True), labels[262800:].reset_index(drop = True))
    training_datas.append([training, validation, testing])

In [206]:
train_dataloader = DataLoader(training_datas[0][0], batch_size = 64)
valid_dataloader = DataLoader(training_datas[0][1], batch_size = 64)

In [207]:
for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 8])
Shape of y: torch.Size([64]) torch.float64


In [225]:
##Create NN
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(8, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        #print(x)
        logits = self.linear_relu_stack(x)
        return logits


In [226]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=8, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=1, bias=True)
  )
)


In [227]:
def weighted_mse_loss(input, target, weight):
    return torch.sum(weight * (input - target) ** 2)

In [228]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [229]:
##Train NN
import time
def train(dataloader, model, loss_fn, optimizer, accs):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X.float())
        loss = loss_fn(pred.squeeze(), y.float())
        #print(pred)
        #print(loss)
        #time.sleep(10)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            #print(loss)
            loss, current = loss.item(), batch * len(X)
            
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            accs.append(loss)
    return accs

In [ ]:
for X, y in train_dataloader:
  print(X[0])
  preds = model(X.float())
  print(preds)
  print(loss_fn(preds.squeeze(), y.float()))
  break

In [231]:
def test(dataloader, model, loss_fn, accs):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X.float())
            test_loss += loss_fn(pred.squeeze(), y.float()).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    accs.append(test_loss)
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
    return accs

In [232]:
epochs = 5
train_accs = []
test_accs = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_accs = train(train_dataloader, model, loss_fn, optimizer, train_accs)
    test_accs = test(valid_dataloader, model, loss_fn, test_accs)
print("Done!")

Epoch 1
-------------------------------
loss: 0.001278  [    0/175200]
loss: 0.007062  [ 6400/175200]
loss: 0.043656  [12800/175200]
loss: 0.021447  [19200/175200]
loss: 0.003299  [25600/175200]
loss: 0.003029  [32000/175200]
loss: 0.004395  [38400/175200]
loss: 0.000430  [44800/175200]
loss: 0.005569  [51200/175200]
loss: 0.052223  [57600/175200]
loss: 0.034987  [64000/175200]
loss: 0.020599  [70400/175200]
loss: 0.000950  [76800/175200]
loss: 0.002924  [83200/175200]
loss: 0.000168  [89600/175200]
loss: 0.003404  [96000/175200]
loss: 0.011558  [102400/175200]
loss: 0.001854  [108800/175200]
loss: 0.000124  [115200/175200]
loss: 0.000532  [121600/175200]
loss: 0.001612  [128000/175200]
loss: 0.000027  [134400/175200]
loss: 0.000153  [140800/175200]
loss: 0.004988  [147200/175200]
loss: 0.000253  [153600/175200]
loss: 0.003149  [160000/175200]
loss: 0.000068  [166400/175200]
loss: 0.003907  [172800/175200]
Test Error: 
 Avg loss: 0.107044 

Epoch 2
-------------------------------
loss:

In [233]:
torch.save(model.state_dict(), "nn_stuff/models/model_0.pth")

In [ ]:
##Plot/record accuracy

In [ ]:
##Test model in testing data

In [ ]:
#Downloading file
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')